In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.quantization
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 데이터셋 & 로더

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('../data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# PTQ(Post-Training static Quantization)
PTQ에서는 모델을 학습한 후에 양자화를 적용합니다.  

정적 양자화에서는 모델을 양자화하기 전에 칼리브레이션(calibration) 단계를 거칩니다. 이는 일부 입력 데이터를 사용하여 모델 내의 활성화 값의 범위를 추정하는 과정입니다. 이를 바탕으로 정적 양자화는 **가중치(weight)**와 **활성화값(activation)**을 모두 정수형(INT8)으로 변환합니다.

### CNN 모델 정의

기본 CNN 구조를 사용하되, 양자화를 위해 `QuantStub`과 `DeQuantStub`을 사용합니다. 이 두 모듈을 사용하면 양자화된 데이터가 네트워크에 들어가고, 부동소수점 값으로 복구됩니다.
- 입력: `QuantStub`에서 양자화되고
- 출력: `DeQuantStub`에서 다시 부동소수점으로 변환

In [ ]:
# CNN 모델 정의
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(1600, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(2)
        
        # 양자화 준비를 위한 QuantStub 및 DeQuantStub 추가
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        # 양자화된 입력값을 받기 위한 QuantStub
        x = self.quant(x)
        
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        
        # 다시 부동 소수점으로 복구하기 위한 DeQuantStub
        x = self.dequant(x)
        return x

In [ ]:
model = CNNModel().to(device)

### 학습 및 추론 함수 정의

In [ ]:
# 모델 학습 함수
def train_model(model, train_loader, optimizer, criterion, device):
    model.train()
    for epoch in range(1, 6):  # 5 epochs
        running_loss = 0.0
        for data, target in tqdm(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch {epoch}, Loss: {running_loss/len(train_loader)}')

In [ ]:
# 평가 함수
def test_model(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)')


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

### 학습
PTQ에서는 학습이 진행된 후 양자화가 수행된다고 가정합니다.

In [ ]:
train_model(model, train_loader, optimizer, criterion, device)

### Inference

In [ ]:
test_model(model, test_loader, criterion, device)

### Quantization

양자화는 CPU 상에서 진행되며, `FBGEMM` Backend를 이용해야 합니다.  

`torch.quantization.get_default_qconfig('fbgemm')`는 CPU에서 INT8 연산을 수행하기 위한 Backend입니다.
  
양자화는 train data로 양자화 준비 과정(칼리브레이션)을 거치며, 이는 모델의 활성화값(activation)의 범위를 추정하기 위해서입니다.

In [ ]:
# 양자화 함수
def quantize_model(model, train_loader):
    # 모델 양자화 준비
    model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
    torch.quantization.prepare(model, inplace=True)
    
    # Calibration을 위한 일부 데이터로 모델 실행
    model.eval()
    with torch.no_grad():
        for data, target in tqdm(train_loader):
            model(data)
            break  # Calibration을 위한 한 batch만 사용
            
    # 양자화 완료
    torch.quantization.convert(model, inplace=True)
    print("Quantization Complete.")

1. qconfig 설정
- `model.qconfig = torch.quantization.get_default_qconfig('fbgemm'))`

- `qconfig`는 양자화의 구성을 정의하는 설정입니다. 여기서는 'fbgemm'을 사용하여 INT8 양자화를 수행하도록 설정합니다.

2. prepare
- `torch.quantization.prepare(model, inplace=True)`
- 모델에 양자화를 적용할 준비를 합니다.
- 양자화를 위한 QuantStub과 DeQuantStub 같은 양자화/비양자화 노드가 삽입되고, 각 레이어에서 양자화 범위를 추적할 수 있게 됩니다.

3. Calibraion

- 모델에 데이터를 통과시켜 활성화값의 범위를 측정합니다.

- 각 레이어에서 활성화값이 차지하는 범위를 추정하며, 추후 양자화에서 활성화값을 INT8로 변환하는 데 중요한 기준으로 사용합니다.

4. convert
- `torch.quantization.convert(model, inplace=True)`
- 모델의 가중치와 활성화값이 모두 8비트 정수(INT8)로 변환되며, 부동소수점 연산 대신 정수형 연산을 수행하는 양자화된 모델이 됩니다.

In [ ]:
model.to('cpu')  # 양자화는 CPU에서 수행
quantize_model(model, train_loader)

In [ ]:
# 양자화된 모델 평가
test_model(model, test_loader, criterion, 'cpu')